In [65]:
# with open('input_test.txt') as f:
with open('input.txt') as f:
    lines = f.readlines()

grid = []
for i,l in enumerate(lines):
    if l.strip() == '':
        break
    row = []
    for c in l.strip():
        if c == '.':
            row += ['.']*2
        elif c == '#':
            row += ['#']*2
        elif c == 'O':
            row += ['[', ']']
        elif c == '@':
            row += ['@', '.']
        else:
            raise Exception()
    grid.append(row)
    
moves = ''
for l in lines[i:]:
    moves += l.strip()

# moves

In [66]:
I = len(grid)
J = len(grid[0])

# Find robot
found = False
pos_robot = (-1,-1)
for i in range(I):
    for j in range(J):
        if grid[i][j] == '@':
            found = True
            pos_robot = (i,j)
            break
    if found:
        break

deltas = {
    '^': (-1,0),
    'v': (1,0),
    '<': (0,-1),
    '>': (0,1),
}

In [67]:
def plus(pos1, pos2):
    return ((pos1[0]+pos2[0]), (pos1[1]+pos2[1]))

def get_symbol(pos):
    return grid[pos[0]][pos[1]]

def set_symbol(pos, symbol):
    grid[pos[0]][pos[1]] = symbol


def move(direction_symbol):
    global pos_robot
    delta = deltas[direction_symbol]
    new_pos = plus(pos_robot, delta)
    if get_symbol(new_pos) == '.':
        set_symbol(pos_robot, '.')
        set_symbol(new_pos,   '@')
        pos_robot = new_pos
    elif get_symbol(new_pos) == '#':
        return
    else:
        # Box is in the way
        if delta[0] == 0:
            # Horizontal movement => easy
            candidate_pos = new_pos
            while True:
                candidate_pos = plus(candidate_pos, delta)
                if get_symbol(candidate_pos) == '.':
                    # Move boxes and robot
                    
                    if delta[1] == 1:
                        start_col = new_pos[1] + 1
                        end_col   = candidate_pos[1]
                    else:
                        start_col = candidate_pos[1]
                        end_col   = new_pos[1] - 1
                    
                    start_box = True
                    for ind in range(start_col, end_col+1):
                        set_symbol((new_pos[0], ind), '[' if start_box else ']')
                        start_box = not start_box
                    
                    set_symbol(pos_robot, '.')
                    set_symbol(new_pos, '@')
                    pos_robot = new_pos
                    return
                elif get_symbol(candidate_pos) == '#':
                    # Cannot move box
                    return
                else:
                    # Another box
                    continue
        else:
            # Vertical movement
            last_box_positions_to_move = set([new_pos])
            if get_symbol(new_pos)=='[':
                last_box_positions_to_move.add(plus(new_pos, (0,1)))
            else:
                last_box_positions_to_move.add(plus(new_pos, (0,-1)))
            
            all_box_positions_to_move = set(last_box_positions_to_move)
            while True:
                new_last_box_positions_to_move = set()
                if len(last_box_positions_to_move) == 0:
                    break
                for pos_box in last_box_positions_to_move:
                    new_pos_box = plus(pos_box, delta)
                    if get_symbol(new_pos_box) == '.':
                        pass
                    elif get_symbol(new_pos_box) == '#':
                        return
                    else:
                        new_last_box_positions_to_move.add(new_pos_box)
                        if get_symbol(new_pos_box)=='[':
                            new_last_box_positions_to_move.add(plus(new_pos_box, (0,1)))
                        else:
                            new_last_box_positions_to_move.add(plus(new_pos_box, (0,-1)))
                
                last_box_positions_to_move = new_last_box_positions_to_move
                all_box_positions_to_move = all_box_positions_to_move | last_box_positions_to_move
            
            ## If this is reached: move boxes and robot
            
            # Get symbols
            symbols_boxes_to_move = dict()
            for pos_box in all_box_positions_to_move:
                symbols_boxes_to_move[pos_box] = get_symbol(pos_box)
            
            for pos_box in all_box_positions_to_move:
                set_symbol(pos_box, '.')
            for pos_box in all_box_positions_to_move:
                set_symbol(plus(pos_box, delta), symbols_boxes_to_move[pos_box])
            
            set_symbol(pos_robot, '.')
            set_symbol(new_pos, '@')
            pos_robot = new_pos


def pretty_print_grid():
    for row in grid:
        print(''.join(row))

# from IPython.display import clear_output
# import time

# pretty_print_grid()
# for s in moves:
#     clear_output(wait=True)
#     print(s)
#     move(s)
#     pretty_print_grid()
#     time.sleep(0.2)

for s in moves:
    move(s)

In [68]:


def get_score(pos):
    return 100*pos[0] + pos[1]

res = 0
for i in range(I):
    for j in range(J):
        if grid[i][j] == '[':
            res += get_score((i,j))

res

1448458